<a href="https://colab.research.google.com/github/ETdan/galaxy-agent/blob/main/bioblend_lookup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m pip install bioblend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 3.1 MB/s eta 0:00:00


In [2]:
from bioblend import galaxy

gi = galaxy.GalaxyInstance(url='https://usegalaxy.org/', key='API_KEY')

hl = gi.histories.get_histories()
con = galaxy.config.ConfigClient(gi)

In [3]:
con.get_config()
con.whoami()

{'model_class': 'User',
 'id': '56b14916fe35b81b',
 'username': 'etdan',
 'email': 'asfawdaniel779@gmail.com',
 'deleted': False,
 'active': True,
 'last_password_change': '2025-03-10T22:54:07.567702'}

In [4]:
# prompt: give me code to send http request

import requests

def send_http_request(url, method='GET', headers=None, data=None, params=None):
    """
    Sends an HTTP request to the specified URL.

    Args:
        url: The URL to send the request to.
        method: The HTTP method to use (GET, POST, PUT, DELETE, etc.). Defaults to GET.
        headers: A dictionary of HTTP headers to include in the request.
        data: The data to send in the request body.
        params: Query parameters to append to the URL.

    Returns:
        A requests.Response object containing the server's response.
        Returns None if there was an error making the request.
    """
    try:
        response = requests.request(method, url, headers=headers, data=data, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Error sending HTTP request: {e}")
        return None

In [5]:
import xml.etree.ElementTree as ET

def extract_inputs_outputs(xml_string):
    try:
        # Parse the XML string
        root = ET.fromstring(xml_string)

        # Extract content from <inputs> tag
        inputs = root.find('inputs')
        inputs_content = ET.tostring(inputs, encoding='unicode', method='xml').strip() if inputs is not None else None

        # Extract content from <outputs> tag (if it exists)
        outputs = root.find('outputs')
        outputs_content = ET.tostring(outputs, encoding='unicode', method='xml').strip() if outputs is not None else None

        return inputs_content, outputs_content
    except ET.ParseError as e:
        return f"Error parsing XML: {e}", None


In [8]:
tools = galaxy.tools.ToolClient(gi)

# tool_list = tools.show_tool(tool_id="upload1")

single_tool = tools.get_tools()

for tool in single_tool[:50]:
  id  = tool.get("id")
  response = send_http_request(f"https://usegalaxy.eu/api/tools/{id}/raw_tool_source")

  if response:
    # print(response.status_code)
    source_xml = response.text
    # print(source_xml)
    inputs, outputs= extract_inputs_outputs(source_xml)

    if inputs:
      tool["xml_input"]=inputs
    if outputs:
      tool["xml_output"]=outputs

  # print(tool)

single_tool

Error sending HTTP request: 404 Client Error: Not Found for url: https://usegalaxy.eu/api/tools/Condense%20characters1/raw_tool_source


[{'model_class': 'Tool',
  'id': 'upload1',
  'name': 'Upload File',
  'version': '1.1.7',
  'description': 'from your computer',
  'labels': [],
  'edam_operations': ['operation_0224'],
  'edam_topics': [],
  'hidden': '',
  'is_workflow_compatible': False,
  'xrefs': [],
  'link': '/tool_runner?tool_id=upload1',
  'min_width': -1,
  'target': 'galaxy_main',
  'panel_section_id': 'get_data',
  'panel_section_name': 'Get Data',
  'form_style': 'regular',
  'xml_input': '<inputs nginx_upload="true">\n    <param name="file_type" type="select" label="File Format" help="Which format? See help below">\n      <options from_parameter="tool.app.datatypes_registry.upload_file_formats" transform_lines="[ &quot;%s%s%s&quot; % ( line, self.separator, line ) for line in obj ]">\n        <column name="value" index="1" />\n        <column name="name" index="0" />\n        <filter type="sort_by" column="0" />\n        <filter type="add_value" name="Auto-detect" value="auto" index="0" />\n      </optio